In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [3]:
def print_rmse(y_train, y_pred):
    mse = mean_squared_error(y_train, y_pred)
    rmse = np.sqrt(mse)
    print(f'MSE : {mse}, RMSE: {rmse}')

In [4]:
import numpy as np
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error

linear_svr = LinearSVR(random_state=42)
linear_svr.fit(scaled_X_train, y_train)

y_pred = linear_svr.predict(scaled_X_train)
print_rmse(y_train, y_pred)

MSE : 0.9641780189948642, RMSE: 0.9819256687727764


In [5]:
from sklearn.svm import SVR

svr = SVR()
svr.fit(scaled_X_train, y_train)

y_pred = svr.predict(scaled_X_train)
print_rmse(y_train, y_pred)

MSE : 0.33613015291858067, RMSE: 0.5797673265358964


In [6]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

params = {
    'gamma': reciprocal(0.001, 0.1),
    'C': uniform(1, 10)
}

svr = SVR()
rcv = RandomizedSearchCV(svr, params, scoring='neg_mean_squared_error', n_jobs=-1, cv=3, random_state=42)
rcv.fit(scaled_X_train, y_train)

RandomizedSearchCV(cv=3, estimator=SVR(), n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000012685328B88>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001268585C708>},
                   random_state=42, scoring='neg_mean_squared_error')

In [7]:
print(f'best score\n mse: {-rcv.best_score_} rmse: {np.sqrt(-rcv.best_score_)}')
print(f'best estimator\n {rcv.best_estimator_}')

best score
 mse: 0.34980808276117986 rmse: 0.591445756397981
best estimator
 SVR(C=4.745401188473625, gamma=0.07969454818643928)


In [8]:
y_pred = rcv.best_estimator_.predict(scaled_X_train)
print_rmse(y_train, y_pred)

MSE : 0.3280453999995986, RMSE: 0.5727524770785357


In [9]:
y_pred = rcv.best_estimator_.predict(scaled_X_test)
print_rmse(y_test, y_pred)

MSE : 0.3515503774395355, RMSE: 0.5929168385528746
